In [1]:
# import_notebooks is a custom python script
# this line makes it possible to import functions from other IPython notebooks
from import_notebooks import *

In [2]:
# Here we import all the functions (*) from these notebooks
from Hainan_Data_Cleaning import *
from time_shift_dataframe import *
from Stratified_Split import *
from Cumulative import *
from Rollingsum import *
from LR_model import *
from Classification_Functions import *
from nn_model_keras import *
from Looping_Function import *

from Utilities import *

importing Jupyter notebook from Hainan_Data_Cleaning.ipynb
importing Jupyter notebook from time_shift_dataframe.ipynb
importing Jupyter notebook from Stratified_Split.ipynb
importing Jupyter notebook from Cumulative.ipynb
importing Jupyter notebook from Rollingsum.ipynb
importing Jupyter notebook from LR_model.ipynb


C:\Users\molin\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


importing Jupyter notebook from Classification_Functions.ipynb
importing Jupyter notebook from nn_model_keras.ipynb


C:\Users\molin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


importing Jupyter notebook from Looping_Function.ipynb
importing Jupyter notebook from Utilities.ipynb


In [5]:
def loop(df, time_shift_min=15, time_shift_max=25, time_bucket_min=1, time_bucket_max=40, num_buckets = 7, output_var = 'RawBiogas',
         run_rf = 0, run_dt = 0, run_log = 0, run_xgb = 0, run_linreg = 0, run_lass = 0, run_ridg = 0, run_nb = 0,
         alpha=0.5):

    test_error_array_rf = []
    test_error_array_dt = []
    test_error_array_log = []
    test_error_array_xgb = []
    test_error_array_linreg = []
    test_error_array_lasso = []
    test_error_array_ridge = []
    test_error_array_nb = []

    for ts in range(time_shift_min,time_shift_max+1):
        print('Code Running for time shift', ts, 'please wait...')
        #print(ts)
        df_ts = timeshift_df(df,ts)
        for tb in range(time_bucket_min, time_bucket_max+1):
            #print(tb)
            df_roll = rollingsum(df_ts,tb)
            np.random.seed(1337)
            x_train_c, y_train_c, x_test_c, y_test_c, bins = test_train_for_classification(df_roll,output_var=output_var, 
                                                                                           num_buckets = num_buckets)
            x_train, y_train, x_test, y_test = test_train_for_regression(df_roll, output_var=output_var)
            if run_rf == 1:
                test_error_rf = RandomForest(x_train_c, y_train_c, x_test_c, y_test_c)[1]
                test_error_array_rf = test_error_array_rf + [test_error_rf]
            if run_dt == 1:
                test_error_dt = DecisionTree(x_train_c, y_train_c, x_test_c, y_test_c)[1]
                test_error_array_dt = test_error_array_dt + [test_error_dt]
            if run_log == 1:
                test_error_log = LogisticModel(x_train_c, y_train_c, x_test_c, y_test_c)[1]
                test_error_array_log = test_error_array_log + [test_error_log]
            if run_xgb == 1:
                test_error_xgb = XGBoost(x_train_c, y_train_c, x_test_c, y_test_c)[1]
                test_error_array_xgb = test_error_array_xgb + [test_error_xgb]
            if run_linreg == 1:
                test_error_linreg = run_lm_quick(x_train, y_train, x_test, y_test)[2]
                test_error_array_linreg = test_error_array_linreg + [test_error_linreg]
            if run_lass == 1:
                test_error_lasso = run_lasso(x_train, y_train, x_test, y_test, alpha=alpha)[2]
                test_error_array_lasso = test_error_array_lasso + [test_error_lasso]
            if run_ridg == 1:
                test_error_ridge = run_ridge(x_train, y_train, x_test, y_test, alpha=alpha)[2]
                test_error_array_ridge = test_error_array_ridge + [test_error_ridge]
            if run_nb == 1:
                test_error_nb = NaiveBayes(x_train_c, y_train_c, x_test_c, y_test_c)[1]
                test_error_array_nb = test_error_array_nb + [test_error_nb]

    print('\n')
    
    if run_rf == 1:
        np_test_error_array_rf = np.array(test_error_array_rf)
        np_test_error_2array_rf = np.reshape(np_test_error_array_rf, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_rf = pd.DataFrame(np_test_error_2array_rf, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for Random Forest at time shift', ts,'is', pd_test_error_rf[ts].max(), 'at a rolling sum bucket of', pd_test_error_rf[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_rf[ts].max()]
        print('The Maximum Accuracy for Random forest is', max(test_error_array)*100, '%', 
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_rf[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
            
    print('\n')
    
    if run_dt == 1:
        np_test_error_array_dt = np.array(test_error_array_dt)
        np_test_error_2array_dt = np.reshape(np_test_error_array_dt, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_dt = pd.DataFrame(np_test_error_2array_dt, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for Decision Tree at time shift', ts,'is', pd_test_error_dt[ts].max(), 'at a rolling sum bucket of', pd_test_error_dt[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_dt[ts].max()]
        print('The Maximum Accuracy for Decision Tree is', max(test_error_array)*100, '%',
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_dt[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
 
    print('\n')
    
    if run_log == 1:
        np_test_error_array_log = np.array(test_error_array_log)
        np_test_error_2array_log = np.reshape(np_test_error_array_log, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_log = pd.DataFrame(np_test_error_2array_log, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []        
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for Logisitic Regression at time shift', ts,'is', pd_test_error_log[ts].max(), 'at a rolling sum bucket of', pd_test_error_log[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_log[ts].max()]
        print('The Maximum Accuracy for Logisitic Regression is', max(test_error_array)*100, '%', 
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_log[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
     
    print('\n')
    
    if run_xgb == 1:
        np_test_error_array_xgb = np.array(test_error_array_xgb)
        np_test_error_2array_xgb = np.reshape(np_test_error_array_xgb, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_xgb = pd.DataFrame(np_test_error_2array_xgb, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []                
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for XG Boost at time shift', ts,'is', pd_test_error_xgb[ts].max(), 'at a rolling sum bucket of', pd_test_error_xgb[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_xgb[ts].max()]
        print('The Maximum Accuracy for XG Boost is', max(test_error_array)*100, '%', 
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_xgb[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
    
    print('\n')
    
    if run_linreg == 1:
        np_test_error_array_linreg = np.array(test_error_array_linreg)
        np_test_error_2array_linreg = np.reshape(np_test_error_array_linreg, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_linreg = pd.DataFrame(np_test_error_2array_linreg, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []                
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for Linreg at time shift', ts,'is', pd_test_error_linreg[ts].max(), 'at a rolling sum bucket of', pd_test_error_linreg[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_linreg[ts].max()]
        print('The Maximum Accuracy for Linreg is', max(test_error_array)*100, '%', 
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_linreg[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
    
    print('\n')
    
    if run_lass == 1:
        np_test_error_array_lasso = np.array(test_error_array_lasso)
        np_test_error_2array_lasso = np.reshape(np_test_error_array_lasso, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_lasso = pd.DataFrame(np_test_error_2array_lasso, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []                
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for Lasso at time shift', ts,'is', pd_test_error_lasso[ts].max(), 'at a rolling sum bucket of', pd_test_error_lasso[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_lasso[ts].max()]
        print('The Maximum Accuracy for Lasso is', max(test_error_array)*100, '%', 
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_lasso[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
    
    print('\n')
    
    if run_ridg == 1:
        np_test_error_array_ridge = np.array(test_error_array_ridge)
        np_test_error_2array_ridge = np.reshape(np_test_error_array_ridge, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_ridge = pd.DataFrame(np_test_error_2array_ridge, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []                
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for Ridge at time shift', ts,'is', pd_test_error_ridge[ts].max(), 'at a rolling sum bucket of', pd_test_error_ridge[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_ridge[ts].max()]
        print('The Maximum Accuracy for Ridge is', max(test_error_array)*100, '%', 
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_ridge[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
    
    print('\n')
    
    if run_nb == 1:
        np_test_error_array_nb = np.array(test_error_array_nb)
        np_test_error_2array_nb = np.reshape(np_test_error_array_nb, (-1, time_shift_max - time_shift_min +1), order='F')
        pd_test_error_nb = pd.DataFrame(np_test_error_2array_nb, index= range(time_bucket_min, time_bucket_max+1), columns = range(time_shift_min, time_shift_max+1))
        test_error_array = []                
        for ts in range(time_shift_min,time_shift_max+1):
            print('The Maximum Accuracy for NB at time shift', ts,'is', pd_test_error_nb[ts].max(), 'at a rolling sum bucket of', pd_test_error_nb[ts].idxmax())
            test_error_array = test_error_array + [pd_test_error_nb[ts].max()]
        print('The Maximum Accuracy for NB is', max(test_error_array)*100, '%', 
              'at time shift', test_error_array.index(max(test_error_array)) + time_shift_min,
              'at a rolling sum bucket of', pd_test_error_nb[test_error_array.index(max(test_error_array)) + time_shift_min].idxmax()
             )
    
    print('\n')
    
    if run_rf == 1:
        return pd_test_error_rf
    if run_dt == 1:
        return pd_test_error_dt 
    if run_log == 1:
        return pd_test_error_log  
    if run_xgb == 1:
        return pd_test_error_xgb  
    if run_linreg == 1:
        return pd_test_error_linreg
    if run_lass == 1:
        return pd_test_error_lasso  
    if run_ridg == 1:
        return pd_test_error_ridge  
    if run_nb == 1:
        return pd_test_error_nb 